# Libraries used

In [1]:
import pickle
import cv2 as cv
import os
from pathlib import Path
import numpy as np
import os
import logging

# Configure logging to display information about the program's execution

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Generates the coordinates for horizontal and vertical grid lines of the game board.
Returns:

    horizontal_lines (list): List of tuples representing horizontal lines.
    
    vertical_lines (list): List of tuples representing vertical lines.

In [3]:
def generate_grid_lines():
    # Generate horizontal lines at specific y-coordinates
    horizontal_lines = [
        [(350, y), (2440, y)] for y in range(370, 2452, 148)
    ]
    # Generate vertical lines at specific x-coordinates
    vertical_lines = [
        [(x, 370), (x, 2440)] for x in range(350, 2600, 150)
    ]
    return horizontal_lines, vertical_lines

# Orders four corner points in a consistent order: top-left, top-right, bottom-right, bottom-left.
Args:

    points (ndarray): Array of four points.
    
Returns:

    rect (ndarray): Ordered array of points.

In [4]:
def order_corner_points(points):
    rect = np.zeros((4, 2), dtype="float32")
    sum_points = points.sum(axis=1)
    diff_points = np.diff(points, axis=1)
    rect[0] = points[np.argmin(sum_points)]      # Top-left point has the smallest sum
    rect[2] = points[np.argmax(sum_points)]      # Bottom-right point has the largest sum
    rect[1] = points[np.argmin(diff_points)]     # Top-right point has the smallest difference
    rect[3] = points[np.argmax(diff_points)]     # Bottom-left point has the largest difference
    return rect


# Extracts and warps the game board from the image.
Args:

    image (ndarray): Image of the game board.
    
    mask (ndarray): Mask of the game board.
    
    kernel_size (tuple): Size of the kernel for morphological operations.
    
    canny_threshold1 (int): First threshold for the hysteresis procedure in Canny edge detection.
    
    canny_threshold2 (int): Second threshold for the hysteresis procedure in Canny edge detection.
    
    width (int): Width of the warped image.
    
    height (int): Height of the warped image.
    
Returns:

    warped (ndarray): Warped image of the game board.

In [5]:
def extract_and_warp_board(image, mask, kernel_size=(3, 4), canny_threshold1=20, canny_threshold2=400, width=2800, height=2800):

    # Erode the mask to reduce noise
    kernel = np.ones(kernel_size, np.uint8)
    mask = cv.erode(mask, kernel)

    # Detect edges using Canny edge detection
    edges = cv.Canny(mask, canny_threshold1, canny_threshold2)

    # Find contours in the edge-detected image
    contours, _ = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    if not contours:
        return image  # Return the original image if no contours are found

    # Find the largest contour, hoping that it is the board
    largest_contour = max(contours, key=cv.contourArea)
    perimeter = cv.arcLength(largest_contour, True)
    approx_poly = cv.approxPolyDP(largest_contour, 0.02 * perimeter, True)

    if len(approx_poly) != 4:
        return image  # Return the original image if the largest contour is not quadrilateral

    # Order the corner points
    corners = order_corner_points(approx_poly.reshape(4, 2))

    # Define the destination points for perspective transformation
    destination_corners = np.array([
        [0, 0],
        [width - 1, 0],
        [width - 1, height - 1],
        [0, height - 1]
    ], dtype="float32")

    # Compute the perspective transform matrix and apply it
    perspective_transform = cv.getPerspectiveTransform(corners, destination_corners)
    warped = cv.warpPerspective(image, perspective_transform, (width, height))
    return warped

# Processes and compares two images of the game board.
Args:

    image1 (ndarray): Image of the game board.
    
    image2 (ndarray): Image of the game board.
    
    low_hsv (tuple): Lower bound of the HSV range for the mask.(defaulted to the best values found for the boards)
    
    high_hsv (tuple): Upper bound of the HSV range for the mask.(defaulted to the best values found for the boards)
    
Returns:

    difference_image (ndarray): Image showing the absolute difference between the two images.

In [6]:
def process_and_compare_images(image1, image2, low_hsv=(14, 0, 0), high_hsv=(120, 255, 255)):
    def create_hsv_mask(image):
        hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
        return cv.inRange(hsv, low_hsv, high_hsv)
    
    # Create masks and extract boards from both images
    mask1 = create_hsv_mask(image1)
    board1 = extract_and_warp_board(image1, mask1)
     
    mask2 = create_hsv_mask(image2)
    board2 = extract_and_warp_board(image2, mask2)
    
    # Compute the absolute difference between the two boards
    difference_image = cv.absdiff(board2, board1)
    return difference_image

# Determines the cells with the maximum intensity in the thresholded image.

Args:

    thresholded_image (ndarray): Thresholded image of the game board.
    
    horizontal_lines (list): List of tuples representing horizontal lines.
    
    vertical_lines (list): List of tuples representing vertical lines.
    
Returns:

    max_row (int): Row index of the cell with maximum intensity.
    
    max_col (int): Column index of the cell with maximum intensity.
    
    row_coords (list): List of row coordinates.
    
    col_coords (list): List of column coordinates.

In [7]:
def find_max_intensity_cell(thresholded_image, horizontal_lines, vertical_lines):
    # Extract row and column coordinates from the grid lines
    row_coords = [line[0][1] for line in horizontal_lines]
    col_coords = [line[0][0] for line in vertical_lines]

    # Extract patches (cells) from the thresholded image
    patches = [
        thresholded_image[row_coords[i]:row_coords[i+1], col_coords[j]:col_coords[j+1]]
        for i in range(len(row_coords) - 1)
        for j in range(len(col_coords) - 1)
    ]

    # Compute mean intensity of each patch
    mean_intensities = np.array([patch.mean() for patch in patches])

    # Find the index of the patch with maximum intensity
    max_index = mean_intensities.argmax()
    num_cols = len(col_coords) - 1
    max_row = max_index // num_cols
    max_col = max_index % num_cols
    return max_row, max_col, row_coords, col_coords

# Applies a mask to an image in the HSV color space.
### Used for preprocessing the pieces and the templates
Args:

    image (ndarray): Image to be masked.
    
    low_hsv (tuple): Lower bound of the HSV range for the mask.(defaulted to the best values found for the pieces)
    
    high_hsv (tuple): Upper bound of the HSV range for the mask.
    (defaulted to the best values found for the pieces)
    
Returns: 

    masked_image (ndarray): Image with the mask applied.

In [9]:
def apply_hsv_mask(image, low_hsv=(0, 0, 45), high_hsv=(255, 255, 255)):
    hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    mask = cv.inRange(hsv, low_hsv, high_hsv)
    masked_image = cv.bitwise_and(image, image, mask=mask)
    return masked_image

# Main function to process images, perform template matching, and calculate scores.
 Args:
    
    training_path_str (str): Path to the directory containing the images.
    
    output_path_str (str): Path to the directory for output annotations.

In [24]:
def main(training_path_str, output_path_str):
    # Load base image and templates from pickle file
    with open('images.pkl', 'rb') as pickle_file:
        data = pickle.load(pickle_file)

    base_image = data.get('base_image')
    if base_image is None:
        logging.error("Failed to load base_image from images.pkl. Exiting.")
        return
    previous_image = base_image

    # Set up training and output paths
    training_path = Path(training_path_str)
    output_path = Path(output_path_str)
    output_path.mkdir(parents=True, exist_ok=True)

    if not training_path.exists():
        logging.error(f"Training path {training_path} does not exist. Exiting.")
        return

    # Get list of image files in the training directory
    image_files = sorted([f for f in training_path.iterdir() if f.suffix.lower() == '.jpg'])
    horizontal_lines, vertical_lines = generate_grid_lines()
    game_number = 1  # Counter for game numbers

    # Extract row and column coordinates for grid cells
    row_coords = [line[0][1] for line in horizontal_lines]
    col_coords = [line[0][0] for line in vertical_lines]
    cell_height = row_coords[1] - row_coords[0]
    cell_width = col_coords[1] - col_coords[0]
    cell_size = (cell_width, cell_height)

    # Load and process templates from pickle data
    templates = []
    template_names = []
    template_dict = data.get('templates')
    if not template_dict:
        logging.error("No templates found in images.pkl. Exiting.")
        return
    for template_name, template_image in template_dict.items():
        if template_image is not None:
            # Apply HSV mask and resize template
            masked_template = apply_hsv_mask(template_image)
            masked_template_gray = cv.cvtColor(masked_template, cv.COLOR_BGR2GRAY)
            resized_template = cv.resize(masked_template_gray, cell_size)
            templates.append(resized_template)
            template_names.append(template_name)

    # Process each image file
    for file_path in image_files:
        annotation_path = output_path / f"{file_path.stem}.txt"

        current_image = cv.imread(str(file_path))
        if current_image is None:
            logging.warning(f"Failed to load image {file_path.name}. Skipping.")
            continue

        # Determine if a new game has started
        current_game_number = int(file_path.stem[0]) if file_path.stem[0].isdigit() else game_number
        if current_game_number != game_number:
            game_number += 1
            previous_image = base_image  # Reset to base_image at new game

        # Compute the difference between the previous and current images
        difference_image = process_and_compare_images(previous_image, current_image)

        # Apply morphological opening to the difference image to remove noise
        kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (13, 13))
        difference_image = cv.morphologyEx(difference_image, cv.MORPH_OPEN, kernel)

        # Convert difference image to grayscale if necessary
        if len(difference_image.shape) == 3:
            difference_gray = cv.cvtColor(difference_image, cv.COLOR_BGR2GRAY)
        else:
            difference_gray = difference_image

        try:
            # Find the cell with the maximum intensity difference
            max_row, max_col, row_coords, col_coords = find_max_intensity_cell(
                difference_gray, horizontal_lines, vertical_lines)
            detected_move = f"{max_row + 1}{chr(max_col + 65)}"  # Convert to human-readable format
        except Exception as e:
            logging.error(f"Error determining cell with max intensity for {file_path.name}: {e}")
            continue

        # Extract the cell image from the current board
        try:
            # Generate mask for current image
            hsv_current = cv.cvtColor(current_image, cv.COLOR_BGR2HSV)
            mask_current = cv.inRange(hsv_current, (14, 0, 0), (120, 255, 255))
            current_board = extract_and_warp_board(current_image, mask_current)

            # Extract the specific cell image
            cell_image = current_board[row_coords[max_row]:row_coords[max_row+1], col_coords[max_col]:col_coords[max_col+1]]
            cell_image_gray = cv.cvtColor(cell_image, cv.COLOR_BGR2GRAY)

            cv.imwrite(f"ok/{file_path.stem}_cell.jpg", cell_image_gray)
            
            cell_image = apply_hsv_mask(cell_image)
            cell_image_gray = cv.cvtColor(cell_image, cv.COLOR_BGR2GRAY)
        except Exception as e:
            logging.error(f"Error extracting cell image for {file_path.name}: {e}")
            continue

        # Perform template matching to identify the piece
        match_scores = []
        for template_img in templates:
            res = cv.matchTemplate(cell_image_gray, template_img, cv.TM_CCOEFF_NORMED)
            match_scores.append(res[0][0])  # Extract the matching score

        # Identify the best matching template
        best_match_index = np.argmax(match_scores)
        best_match_name = template_names[best_match_index]
        cleaned_name = best_match_name.split()[0]  # Clean up the template name if necessary

        # Write the detected move and piece to the annotation file
        with open(annotation_path, 'w') as txt_file:
            txt_file.write(f"{detected_move} {cleaned_name}\n")

        # Update previous_image for the next iteration
        previous_image = current_image

    # --- Score Logic Starts Here ---

    # Define coordinates for double and triple multiplier cells on the grid
    double_multiplier_cells = [
        (1, 1), (2, 2), (3, 3), (4, 4), (9, 9),
        (10, 10), (11, 11), (12, 12),
        (1, 12), (2, 11), (3, 10), (4, 9),
        (9, 4), (10, 3), (11, 2), (12, 1)
    ]

    triple_multiplier_cells = [
        (0, 0), (0, 13), (13, 0), (13, 13),
        (6, 0), (7, 0), (0, 6), (0, 7),
        (13, 6), (13, 7), (6, 13), (7, 13)
    ]

    # Initialize the game grid
    grid_size = 14
    game_grid = [[" " for _ in range(grid_size)] for _ in range(grid_size)]

    game_grid[6][6] = "1"
    game_grid[7][7] = "2"
    game_grid[6][7] = "3"
    game_grid[7][6] = "4"

    total_score = 0  # Initialize total score

    # Open and read the turns file
    turns_file_path = training_path / '4_turns.txt'
    if not turns_file_path.exists():
        logging.error(f"Turns file {turns_file_path} does not exist. Exiting.")
        return
    with open(turns_file_path, 'r') as file:
        turns = file.readlines()

    # Extract the turn numbers from the turns file
    turn_numbers = [int(line.split()[1]) for line in turns]

    # Calculate the differences between consecutive turn numbers
    differences = [turn_numbers[i+1] - turn_numbers[i] for i in range(len(turn_numbers) - 1)]
    differences.append(51 - turn_numbers[-1])  # Add the remaining turns

    file_list = differences
    folder_path = output_path  # 'annotations' folder

    # Function to convert positions like "9H" into grid indices
    def position_to_indices(position):
        """
        Converts a position string (e.g., "9H") to zero-based grid indices.
        Args:
            position (str): Position string.
        Returns:
            row (int): Row index.
            col (int): Column index.
        """
        row = int(position[:-1]) - 1  # Convert row number to zero-based index
        col = ord(position[-1].upper()) - ord('A')  # Convert column letter to zero-based index
        return row, col

    # Function to check for valid equations and calculate score
    def calculate_equations_and_score(game_grid, row, col, piece_value_str):
        """
        Calculates the score for placing a piece based on adjacent cells and multipliers.
        Args:
            game_grid (list): The game grid.
            row (int): Row index where the piece is placed.
            col (int): Column index where the piece is placed.
            piece_value_str (str): The value of the piece as a string.
        Returns:
            local_score (int): Score obtained from this move.
        """
        nonlocal total_score
        piece_value = int(piece_value_str)  # Convert piece value to integer
        local_score = 0  # Initialize local score
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Directions: up, down, left, right

        for dr, dc in directions:
            # Check two cells in the current direction
            r1, c1 = row + dr, col + dc
            r2, c2 = row + 2 * dr, col + 2 * dc
            if 0 <= r1 < grid_size and 0 <= c1 < grid_size and 0 <= r2 < grid_size and 0 <= c2 < grid_size:
                try:
                    val1 = int(game_grid[r1][c1])
                    val2 = int(game_grid[r2][c2])
                except ValueError:
                    continue  # Skip if cells do not contain integers

                # Check if placing the piece creates a valid equation
                if val1 + val2 == piece_value or val1 - val2 == piece_value or \
                   val1 * val2 == piece_value or (val2 != 0 and val1 // val2 == piece_value):
                    # Valid equation found
                    local_score += piece_value

        # Apply multipliers based on cell position
        if (row, col) in double_multiplier_cells:
            local_score *= 2
        elif (row, col) in triple_multiplier_cells:
            local_score *= 3

        # Ensure the minimum score is at least the piece value
        if local_score == 0:
            local_score = piece_value

        # Update the total score
        total_score += local_score
        return local_score

    # Open a file to write the updated turns with calculated scores
    output_file = output_path / '4_scores.txt'
    with open(output_file, 'w') as outfile:
        file_counter = 1  # Counter for the file names
        for turn, count in zip(turns, file_list):
            turn_score = 0  # Score for the current turn
            for _ in range(count):
                # Construct the file name for the annotation
                file_name = f"4_{file_counter:02}.txt"  # Format as 1_01.txt, 1_02.txt, etc.
                file_path = folder_path / file_name

                # Parse the annotation file if it exists
                if file_path.exists():
                    with open(file_path, 'r') as file:
                        content = file.read().strip()
                        position, piece_value_str = content.split()  # Split into position and piece value
                        row, col = position_to_indices(position)  # Convert position to indices

                        # Place the piece on the grid
                        game_grid[row][col] = piece_value_str

                        # Calculate score for this move
                        step_score = calculate_equations_and_score(game_grid, row, col, piece_value_str)
                else:
                    logging.warning(f"File {file_path} does not exist. Skipping.")
                    step_score = 0  # Assign zero score if file does not exist

                file_counter += 1  # Increment the file counter
                turn_score += step_score  # Add to the turn score

            # Write the turn information and score to the output file
            player, turn_number = turn.strip().split()
            outfile.write(f"{player} {turn_number} {turn_score}\n")

    print(f"Scores saved to {output_file}")




# Run the main function with the images folder and annotations folder.

The images folder should contain the images of the game board that need to be analyzed (including the turns file), and the annotations folder (output folder) is where the score file and the position files will be generated

In [25]:
if __name__ == "__main__":
    main('evaluare4/', 'annot4/')

Scores saved to annot4\4_scores.txt
